In [41]:
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
from logging import getLogger
from dateutil import rrule

import pandas as pd
import numpy as np
import numpy_financial as npf


from pgloader.engine import PostgresEngineCreator
from pgloader.ledger import Ledger
from pgloader.tables import available_tables
from pgloader.utils import create_parser, setup_logger

In [42]:
period = rrule.rrule(rrule.MONTHLY, dtstart=date(2023, 1,1), bymonthday=1, count=31)

In [43]:
INTERVAL = 6
INTEREST_RATE = 0.06
YEARS = 30

In [44]:
le = Ledger("~/Nextcloud/Finanzas/Beans/main.bean")

# Data collection

Leer income y expenses en los ultimos `INTERVAL` meses.

El resultado es el ahorro por mes en `save_per_month`. 

## Saving rate 

In [45]:
today = datetime.today() 
date_until = date(today.year, today.month, 1)
date_from = date_until - relativedelta(months=+INTERVAL)

### Expenses

In [46]:
query_expenses = f"""
SELECT
    SUM(CONVERT(POSITION, 'USD', DATE)) AS amount_usd
WHERE account ~ '^Expenses'
    AND DATE < DATE('{date_until}')
    AND DATE >= DATE('{date_from}')
"""

In [47]:
_, res_ex = le.run_query(query_expenses)

In [48]:
expenses_usd = res_ex[0].amount_usd.get_only_position().units.number

### Income

In [49]:
query_income = f"""
SELECT
    SUM(CONVERT(POSITION, 'USD', DATE)) AS amount_usd
WHERE account ~ ':Job:'
    AND DATE < DATE('{date_until}')
    AND DATE >= DATE('{date_from}')
"""

In [50]:
_, res_in = le.run_query(query_income)
income_usd = res_in[0].amount_usd.get_only_position().units.number

### Save rate

In [66]:
print(income_usd, expenses_usd)

-12600 4288.345448716470042195326623


In [51]:
save_per_month = float((abs(income_usd) - abs(expenses_usd)) / INTERVAL)

## Initial capital

### Assets

In [52]:
query_assets = """
SELECT SUM(CONVERT(POSITION, 'USD', DATE)) AS amount_usd
WHERE Account ~ '^Assets' 
"""

In [53]:
_, res_ass = le.run_query(query_assets)
assets_usd = res_ass[0].amount_usd.get_only_position().units.number

### Liabilities

In [54]:
query_liabilities = """
SELECT SUM(CONVERT(POSITION, 'USD', DATE)) AS amount_usd
WHERE Account ~ '^Liabilities' 
"""

In [55]:
t, res_liab = le.run_query(query_liabilities)
liabilities_usd = res_liab[0].amount_usd.get_only_position().units.number

In [56]:
initial_amount = float(assets_usd - liabilities_usd)

# Analysis

In [57]:
# Monthly interest rate
monthly_interest_rate = INTEREST_RATE / 12

months = 12 * YEARS + 1

# Initialize an array to store the date and future value
future_values_dates = np.zeros((YEARS * 12 + 1, 2), dtype='object')

In [58]:
df = pd.DataFrame(future_values_dates, columns=['Date', 'Future Value'])

In [59]:
npf.fv(monthly_interest_rate, 12 * YEARS, save_per_month, initial_amount)

-1540111.3555074986

In [60]:
df

,Date,Future Value
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
356,0,0
357,0,0
358,0,0
359,0,0


In [65]:
print(monthly_interest_rate)

0.005


In [61]:
dates = [date_until + relativedelta(months=+month) for month in range(months)]

# Calculate future value for the initial amount
#fv_initial_amount = npf.fv(rate=monthly_interest_rate, nper=np.arange(1, months + 1), pmt=0, pv=-initial_amount)

# Calculate future value for monthly contributions
fv_monthly_contributions = np.array([npf.fv(rate=monthly_interest_rate, nper=month, pmt=-save_per_month, pv=-initial_amount) for month in range(1, months + 1)])

# Total future value combines both the initial amount's growth and the contributions
total_fv = fv_monthly_contributions #+  fv_initial_amount

# Create a pandas DataFrame
df_fv = pd.DataFrame({'Date': dates, 'Future Value': total_fv})

# Display the first few rows of the DataFrame
print(df_fv.head())

         Date  Future Value
0  2024-03-01  26179.307868
1  2024-04-01  27695.480166
2  2024-05-01  29219.233325
3  2024-06-01  30750.605250
4  2024-07-01  32289.634035


In [62]:
print(df_fv)

           Date  Future Value
0    2024-03-01  2.617931e+04
1    2024-04-01  2.769548e+04
2    2024-05-01  2.921923e+04
3    2024-06-01  3.075061e+04
4    2024-07-01  3.228963e+04
..          ...           ...
356  2053-11-01  1.513124e+06
357  2053-12-01  1.522075e+06
358  2054-01-01  1.531071e+06
359  2054-02-01  1.540111e+06
360  2054-03-01  1.549197e+06

[361 rows x 2 columns]


In [63]:
type(res_in[0])

beancount.query.query_execute.ResultRow

In [64]:
print(t[0])

('amount_usd', <class 'beancount.core.inventory.Inventory'>)
